In [1]:
import pandas as pd
import datetime
import os
from haversine import haversine
import numpy as np
import json
zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/updated_zip_centers_JL_2019-05-23.json","r"))
os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q3/Planner_request/Faith_zips_for_SOTF_2_stores_20191003'

In [2]:
list_2_stores=["1671","5268"]

store_list=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20191001-134050-499.txt",
                        dtype=str,sep="|")
store_list=store_list[store_list['location_id'].isin(list_2_stores)]
store_list=store_list[['location_id','state_nm','latitude_meas','longitude_meas','zip_cd']]
store_list['zip_cd']=store_list['zip_cd'].apply(lambda x: x.split("-")[0].zfill(5))
store_list['latitude_meas']=store_list['latitude_meas'].astype(float)
store_list['longitude_meas']=store_list['longitude_meas'].astype(float)
store_list=store_list.rename(columns={"zip_cd":"store_zip"})

In [3]:
store_list

,location_id,state_nm,latitude_meas,longitude_meas,store_zip
417,1671,OH,39.629065,-84.170140,45459
1353,5268,OH,39.351079,-84.542525,45014


In [4]:
store_zip=pd.ExcelFile("/home/jian/Projects/Big_Lots/New_TA/TA_created_in_201906/final_output_20190718/BL_final_TA_updated_JL_2019-07-18.xlsx")
store_zip=store_zip.parse("view_by_store",dtype=str)
store_zip=store_zip[store_zip['location_id'].isin(list_2_stores)]
store_zip

,TA_num,ta_name,location_id,address_line_1,address_line_2,zip_cd,city_nm,state_nm,latitude_meas,longitude_meas,DMA,CTY,all_trans_P_zips,all_trans_S_zips,zips_in_10,trans_P_zips_70_within_TA,trans_S_zips_70_within_TA
126,9,multiple_9,5268,4613 DIXIE HWY,nan,45014,FAIRFIELD,OH,39.351079,-84.542525,{'CINCINNATI'},{'BUTLER'},"['45014', '45011']","['45015', '45013', '45240', '45231', '45069', ...",[],"['45011', '45014']","['45069', '45240', '45231', '45013', '45044', ..."
313,29,multiple_29,1671,359 MIAMISBURG CENTERVILLE RD,nan,45459,DAYTON,OH,39.629065,-84.17014,{'DAYTON'},"{'GREENE', 'MONTGOMERY'}","['45458', '45459', '45342', '45066']","['45449', '45429', '45440', '45420', '45068', ...",[],"['45066', '45458', '45459', '45342']","['45429', '45417', '45440', '45420', '45449', ..."


In [5]:
df_store_zip=pd.DataFrame()

for ind,row in store_zip.iterrows():
    store_number=row['location_id']
    P_zips=eval(row['all_trans_P_zips'])
    S_zips=eval(row['all_trans_S_zips'])
    # zips_in_10=eval(row['zips_in_10']) # No 10 zips from the df above
    
    df_P=pd.DataFrame({"trans_flag":["P"]*len(P_zips),"zip_cd":P_zips},index=[store_number]*len(P_zips))
    df_S=pd.DataFrame({"trans_flag":["S"]*len(S_zips),"zip_cd":S_zips},index=[store_number]*len(S_zips))
    df_store_zip=df_store_zip.append(df_P).append(df_S)
df_store_zip=df_store_zip.reset_index().rename(columns={"index":"location_id"})
df_store_zip.shape

(26, 3)

In [6]:
df_output=pd.merge(store_list,df_store_zip,on="location_id",how="left")
df_output.shape

(26, 7)

In [7]:
df_output['dist_miles']=np.nan
for ind,row in df_output.iterrows():
    dist=haversine((row['latitude_meas'],row['longitude_meas']),zip_centers[row['zip_cd']],miles=True)
    df_output.loc[ind,'dist_miles']=dist

In [10]:
writer=pd.ExcelWriter("./BL_zips_for_SOTF_1671_5268_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_output.to_excel(writer,"zips_for_2_SOTF",index=False)
writer.save()